In [2]:
import numpy as np
from tqdm import tqdm
import glob
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu, ttest_ind, zscore, pearsonr, wilcoxon
import pybedtools as pbt
from scipy.integrate import simps
from IPython.display import clear_output
import pandas as pd
from ipyparallel import Client

plt.style.use('fivethirtyeight')
def clean_axis(ax):
    ax.xaxis.set_tick_params(labelsize=9)
    ax.yaxis.set_tick_params(labelsize=9)
    for i in ['top', 'bottom', 'left', 'right']:
        ax.spines[i].set_visible(False)
    ax.grid(which='major', linestyle='--', alpha=0.5)
    return True

DVIEW = Client()[:]
DVIEW.block=True
len(DVIEW)

25

In [6]:
def calc_stats(chrom):
    import numpy as np
    
    def approx_motifs(a):
        ctr = 0
        cctr = {}
        for i in a:
            if i not in cctr:
                cctr[i] = 0
            cctr[i]+=1
            if cctr[i] == 12:
                cctr[i] = 0
                ctr+=1
        return ctr
    
    num_g = {'positive': [], 'negative': []}
    motifs = {'positive': [], 'negative': []}
    cut_offs = [0.5, 0.1, 0.01, 0.001, 1e-5, 1e-10, 1e-20]
    indir = '/home/parashar/scratch/quadcomb/data/QG_expectation_scores_corrected_values'
    for strand in ['positive', 'negative']:
        fn_in = '%s/%s_%s.npy' % (indir, chrom, strand)
        a = np.load(fn_in)
        nz = a[a > 0]
        for cf in cut_offs:
            b = nz[nz > cf]
            num_g[strand].append(len(b))
            motifs[strand].append(approx_motifs(b))
    return num_g, motifs

chroms = ['chr'+str(x) for x in range(1,23)] + ['chrX', 'chrY']
res = DVIEW.map_sync(calc_stats, chroms)

In [13]:
cut_offs = [0.5, 0.1, 0.01, 0.001, 1e-5, 1e-10, 1e-20],

for i in res:
    
        print (sum(i[0]['positive'])
    print (i[1])
    break
    

32782002
{'positive': [344, 29445, 199785, 381757, 619108, 746335, 747037], 'negative': [345, 29989, 199708, 382016, 618997, 745993, 746667]}


In [179]:
for chrom in ['chrX', 'chrY']:
    print ("\t".join(map(str, num_g[chrom]['positive'])))
    print ("\t".join(map(str, num_g[chrom]['negative'])))    

1458	117138	800996	1634137	2928752	3760098	3767169
1509	113974	791444	1626613	2907893	3732872	3739546
180	14792	119851	256486	464937	596068	597115
186	15077	118619	253303	458029	585317	586417


In [151]:
for i in range(len(cut_offs)):
    v = 0
    for chrom in chroms:
        v+=motifs[chrom]['positive'][i]
        v+=motifs[chrom]['negative'][i]
    print (cut_offs[i], v)

0.5 7205
0.1 622933
0.01 4316976
0.001 8392890
1e-05 13955355
1e-10 17123821
1e-20 17140281


In [184]:
from collections import Counter

In [236]:
ctrs = {}
spans = {}
gs = {}
for chrom in chroms:
    ctr = 0
    g4_bed = '/home/parashar/scratch/quadruplexes/hg19/g3_50_3_%s_nov.bed' % chrom
    ctr = 0
    span = 0
    g = 0
    with open(g4_bed) as h:
        for l in h:
            c = l.rstrip('\n').split('\t')
            s = int(c[2]) - int(c[1])
            span+=s
            ctr+=1
            g4 = c[5].upper()
            if c[4][0] == "G":
                g+= g4.count('G')
            else:
                g+= g4.count('C')
    gs[chrom] = g
    ctrs[chrom] = ctr
    spans[chrom] = span
    print (chrom, ctr, span, g)

chr1 1543900 178480655 53261734
chr2 1499943 171133452 50281568
chr3 1196799 135765093 39488479
chr4 1042507 117234654 33742095
chr5 1075474 121939314 35450520
chr6 1018596 115566198 33690818
chr7 1004566 115339960 34147490
chr8 895238 102037948 29908455
chr9 800357 92223280 27428076
chr10 892461 102853135 30548390
chr11 884738 102064825 30438106
chr12 854457 97892269 28900139
chr13 539500 60865702 17614650
chr14 577109 66325987 19617637
chr15 575730 66554369 19865248
chr16 623707 73514709 22451185
chr17 633825 75205538 23220888
chr18 456737 51841529 15151881
chr19 501522 60812949 19227288
chr20 457057 53568634 16270220
chr21 224289 25891231 7733076
chr22 308936 37336382 11746555
chrX 919083 104151753 30070154
chrY 162061 18592426 5355816


In [197]:
sum(list(ctrs.values()))

537072052

In [211]:
x = np.linspace(1e-10, 0.6, 1000)
x[18]

0.010810810909009009

In [216]:
np.argmax(tpr-fpr)

16

In [217]:
x[15:18]

array([ 0.00900901,  0.00960961,  0.01021021])